In [1]:
data_root = '/home/scott/Documents/SPEC_Spider'

In [2]:
!tree /home/scott/Documents/SPEC_Spider

/home/scott/Documents/SPEC_Spider
├── cpu
│   ├── cpu2006
│   │   ├── SPECfp.csv
│   │   ├── SPECfp_rate.csv
│   │   ├── SPECint.csv
│   │   └── SPECint_rate.csv
│   └── cpu2017
│       ├── CFP2017_rate.csv
│       ├── CFP2017_speed.csv
│       ├── CINT2017_rate.csv
│       └── CINT2017_speed.csv
├── java
│   ├── jbb2015
│   │   ├── SPECjbb2015-Composite.csv
│   │   ├── SPECjbb2015-Distributed.csv
│   │   └── SPECjbb2015-MultiJVM.csv
│   └── jvm2008
│       └── jvm2008.csv
├── jbb2015.csv
└── power
    └── ssj2008.csv

7 directories, 14 files


In [14]:
import pandas as pd
import numpy as np
import re

In [4]:
c2017_rfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_rate.csv")
c2017_sfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_speed.csv")
c2017_rint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_rate.csv")
c2017_sint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_speed.csv")

c2006_rfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp_rate.csv")
c2006_sfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp.csv")
c2006_rint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint_rate.csv")
c2006_sint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint.csv")

j2015_com = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Composite.csv")
j2015_dis = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Distributed.csv")
j2015_mul = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-MultiJVM.csv")

jvm2008 = pd.read_csv(f"{data_root}/java/jvm2008/jvm2008.csv")
ssj2008 = pd.read_csv(f"{data_root}/power/ssj2008.csv")

Get all the hardware vendors to clean

In [156]:
def clean_vendor(vendor):
    def _clear(pattern):
        return re.sub(re.compile(pattern, re.IGNORECASE), '', vendor)
    patterns = [
        ' *[(].*[)]',
        ',* Ltd\.*$|,* Inc\.*$',
        ',* Co\.*$|,* Corporation\.*$|,* Corp\.*$| Incoporated$| Incorporated$',
        ' International$',
        ' Computer[s]*$',
        ' Technology$'
    ]
    for pattern in patterns:
        vendor = _clear(pattern)
    
    replace_pairs = [
        ('^Huawei', 'Huawei'),
        ('^ASUS', 'ASUS'),
        ('^Hewlett[ -]*Packard', 'Hewlett Packard'),
        ('^Inspur', 'Inspur'),
        ('H3C', 'H3C'),
        ('^Giga[ -]*byte', 'Gigabyte'),
        ('^Fujitsu', 'Fujitsu'),
        ('^Hitachi', 'Hitachi'),
        ('^Lenovo', 'Lenovo'),
        ('^Quanta', 'Quanta'),
        ('^Super[ -]*Micro', 'SuperMicro'),
        ('^UNIWIDE', 'Uniwide'),
        ('^Wizbrain', 'Wizbrain')
    ]
    for pair in replace_pairs:
        if len(re.findall(re.compile(pair[0], re.IGNORECASE), vendor)):
            vendor = pair[1]
    
    return vendor

In [157]:
c2017_rfp['Hardware Vendor'].apply(lambda x: clean_vendor(x)).unique()

array(['xFusion', 'ZTE', 'Yadro', 'KTNF', 'SuperMicro', 'Sugon',
       'ScaleMP', 'Quanta', 'Huawei', 'Oracle', 'Sun Microsystems',
       'Nokia', 'H3C', 'Tyrone Systems', 'Nettrix', 'NEC',
       'M Computers s.r.o.', 'Lenovo', 'Intel', 'ASUS', 'Inspur',
       'Hitachi', 'Hewlett Packard', 'Gigabyte', 'Fujitsu',
       'Epsylon Sp. z o.o. Sp. Komandytowa', 'Dell', 'Cisco Systems',
       'Altos Computing', 'ATOS'], dtype=object)

In [159]:
print('-----CPU2017-------')
print(c2017_rfp.columns.tolist())
print('-----CPU2006-------')
print(c2006_rfp.columns.tolist())
print('-----jbb2015-------')
print(j2015_com.columns.tolist())
print('-----jvm2008-------')
print(jvm2008.columns.tolist())
print('-----ssj2008-------')
print(ssj2008.columns.tolist())

-----CPU2017-------
['Suite', 'Hardware Vendor', 'System Name', 'Metric', 'Baseline', 'License', 'Test Sponsor', 'Tested By', 'Test Date', 'HW Avail', 'SW Avail', 'CPU Name', 'Max MHz', 'Nominal', 'Enabled', 'Orderable', 'L1', 'L2', 'L3', 'Other Cache', 'Memory', 'Storage', 'Other HW', 'OS', 'Compiler', 'Parallel', 'Firmware', 'File System', 'System State', 'Base Pointers', 'Peak Pointers', 'Other SW', '503.bwaves_r', '507.cactuBSSN_r', '508.namd_r', '510.parest_r', '511.povray_r', '519.lbm_r', '521.wrf_r', '526.blender_r', '527.cam4_r', '538.imagick_r', '544.nab_r', '549.fotonik3d_r', '554.roms_r', 'Base Copies', 'URL Suffix']
-----CPU2006-------
['Suite', 'Hardware Vendor', 'System Name', 'Metric', 'Baseline', 'License', 'Test Sponsor', 'Tested By', 'Test Date', 'HW Avail', 'SW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'FPU', 'CPU(s) enabled', 'CPU(s) orderable', 'Primary Cache', 'Secondary Cache', 'L3 Cache', 'Other Cache', 'Memory', 'Disk Subsystem', 'Other Hardware', '

In [51]:
set(c2017_rfp.columns.tolist()) - set(c2017_sfp.columns.tolist())

{'503.bwaves_r',
 '507.cactuBSSN_r',
 '508.namd_r',
 '510.parest_r',
 '511.povray_r',
 '519.lbm_r',
 '521.wrf_r',
 '526.blender_r',
 '527.cam4_r',
 '538.imagick_r',
 '544.nab_r',
 '549.fotonik3d_r',
 '554.roms_r',
 'Base Copies'}

In [49]:
c2017_rfp['OS'].unique()

array(['Red Hat Enterprise Linux release 8.4 (Ootpa) 4.18.0-305.el8.x86_64',
       'Red Hat Enterprise Linux release 8.3 (Ootpa) 4.18.0-240.el8.x86_64',
       'SUSE Linux Enterprise Server 15 SP2 5.3.18-22-default',
       'Red Hat Enterprise Linux release 8.0 (Ootpa) 4.18.0-80.el8.x86_64',
       'Red Hat Enterprise Linux release 8.2 (Ootpa) 4.18.0-193.el8.x86_64',
       'RHEL 7.2 Red Hat Enterprise Linux Server release 7.4 (Maipo) 3.10.0-693.el7.ppc64le',
       'RHEL 7.2 Red Hat Enterprise Linux Server release 7.2 (Maipo) 3.10.0-327.el7.ppc64le',
       'CentOS Linux release 8.2.2004 (Core) 4.18.0-193.el8.x86_64',
       'Ubuntu 20.04.4 LTS Kernel 5.4.0-104-generic',
       'Ubuntu 20.04.3 LTS Kernel 5.4.0-99-generic',
       'Ubuntu 20.04.3 LTS Kernel 5.4.0-100-generic',
       'SUSE Linux Enterprise Server 15 SP3 5.3.18-57-default',
       'Red Hat Enterprise Linux release 8.4 Kernel 4.18.0-305.el8.x86_64',
       'Red Hat Enterprise Linux 8.4 Kernel 4.18.0-305.el8.x86_64',
   

In [17]:
hw_vendors = set(c2017_rfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2017_sfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2017_rint['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2017_sint['Hardware Vendor'].unique().tolist())

hw_vendors = set(c2006_rfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_sfp['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_rint['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_sint['Hardware Vendor'].unique().tolist())
hw_vendors |= set(c2006_sint['Hardware Vendor'].unique().tolist())

hw_vendors |= set(j2015_com['Vendor'].unique().tolist())
hw_vendors |= set(j2015_mul['Vendor'].unique().tolist())
hw_vendors |= set(j2015_dis['Vendor'].unique().tolist())

hw_vendors |= set(jvm2008['HW vendor'].unique().tolist())

hw_vendors |= set(ssj2008['Hardware Vendor'].unique().tolist())

hw_vendors_df = pd.DataFrame(list(hw_vendors), columns=['Vendor'])

In [18]:
hw_vendors_df['Vendor'].values

array(['GIGA-BYTE Technology Co. Ltd.  (Test Sponsor: Advanced Micro Devices)',
       'FORMAT', 'UNIWIDE Technologies Inc.', 'Quanta Cloud Technology',
       'NEC Corporation', 'Colfax International', 'Apple Computer, Inc.',
       'UNIWIDE', 'Bull SAS', 'Cisco Systems', 'Hewlett-Packard Company',
       'Acer Incorporated', 'Nettrix', 'Huawei Company',
       'Oracle Corporation',
       'Supermicro  (Test Sponsor: Advanced Micro Devices)',
       'Hewlett-Packard Company  (Test Sponsor: Advanced Micro Devices)',
       'Hitachi, Ltd.', 'MSI  (Test Sponsor: Advanced Micro Devices)',
       'Supermicro  (Test Sponsor: Intel Corporation)',
       'Microsoft Corporation  (Test Sponsor: Intel Corporation)',
       'ClearCube Technology', 'TAEJIN T&S', 'Wizbrain', "Plat'Home",
       'DEPO Computers', 'Maxdata AG  (Test Sponsor: Maxdata)',
       'SGI  (Test Sponsor: Japan Advanced Institute of Science and Technology)',
       'Nettars tech', 'Cryo Performance Computing Ltd',
       'IWI

In [155]:
import re

def clean_vendor(vendor):
    def _clear(pattern):
        return re.sub(re.compile(pattern, re.IGNORECASE), '', vendor)
    patterns = [
        ' *[(].*[)]',
        ',* Ltd\.*$|,* Inc\.*$',
        ',* Co\.*$|,* Corporation\.*$|,* Corp\.*$| Incoporated$| Incorporated$',
        ' International$',
        ' Computer[s]*$',
        ' Technology$'
    ]
    for pattern in patterns:
        vendor = _clear(pattern)
    
    replace_pairs = [
        ('^Huawei', 'Huawei'),
        ('^ASUS', 'ASUS'),
        ('^Hewlett[ -]*Packard', 'Hewlett Packard'),
        ('^Inspur', 'Inspur'),
        ('H3C', 'H3C'),
        ('^Giga[ -]*byte', 'Gigabyte'),
        ('^Fujitsu', 'Fujitsu'),
        ('^Hitachi', 'Hitachi'),
        ('^Lenovo', 'Lenovo'),
        ('^Quanta', 'Quanta'),
        ('^Super[ -]*Micro', 'SuperMicro'),
        ('^UNIWIDE', 'Uniwide'),
        ('^Wizbrain', 'Wizbrain')
    ]
    for pair in replace_pairs:
        if len(re.findall(re.compile(pair[0], re.IGNORECASE), vendor)):
            vendor = pair[1]
    
    return vendor

In [154]:
np.sort(hw_vendors_df['Vendor'].apply(lambda x: clean_vendor(x)).unique())

array(['3Score', 'ACTION S.A.', 'ASUS', 'ATEC', 'Acer', 'Apple',
       'BOXX Technologies', 'Bull SAS', 'Cisco Systems', 'ClearCube',
       'Clevo', 'Colfax', 'Cryo Performance Computing', 'DEPO', 'Dell',
       'E4 Computer Engineering S.p.A.', 'EXO S.A.', 'FORMAT',
       'Five9 Network Systems', 'Fujitsu', 'Gigabyte', 'H3C',
       'Hewlett Packard', 'Hitachi', 'Huawei', 'Hypertechnologies Ciara',
       'IBM', 'IWILL', 'Incom S.A.', 'Inspur', 'Intel', 'Itautec',
       'J.P Sa Couto S.A.', 'KTNF', 'LSDtech', 'LTechKorea', 'Lenovo',
       'M Computers s.r.o.', 'MSI', 'Maxdata AG', 'Microsoft', 'NEC',
       'NTT System S. A.', 'Nettars tech', 'Nettrix', 'Nokia', 'Oracle',
       'PC Factory S.A.', "Plat'Home", 'Powerleader', 'Quanta', 'SGI',
       'ScaleMP', 'Shuttle', 'Sugon', 'Sun Microsystems', 'SuperMicro',
       'TAEJIN T&S', 'Tyan', 'Unisys', 'Uniwide', 'Verari Systems',
       'Wipro Limited', 'Wizbrain', 'XSLAB', 'YOYOtech', 'Yadro',
       'ZT Systems', 'ZTE'], dtype=o

In [26]:
c2017_df['Hardware Vendor'].apply(lambda x: clean_hw_vendor(x)).unique()

array(['xFusion', 'ZTE', 'Yadro', 'KTNF', 'Supermicro', 'Sugon',
       'ScaleMP', 'Quanta Cloud Technology', 'Huawei', 'Oracle',
       'Sun Microsystems', 'Nokia', 'New H3C Technologies',
       'Tyrone Systems', 'Nettrix', 'NEC', 'Mputers s.r.o.',
       'Lenovo Global Technology', 'Intel', 'ASUSTeKputer', 'Inspur',
       'Hitachi Vantara', 'Hewlett Packard Enterprise', 'H3C',
       'GIGA-BYTE TECHNOLOGY', 'Fujitsu',
       'Epsylon Sp. z o.o. Sp. Komandytowa', 'Dell', 'Cisco Systems',
       'Altosputing', 'ATOS'], dtype=object)

In [31]:
c2017_df['Metric'].apply(lambda x: re.sub('®', '', x)).unique()

array(['SPECrate2017_fp_base'], dtype=object)

In [46]:
c2017_df['System State'].unique()

array(['Run level 3 (multi-user)', 'Run level 5 (multi-user)',
       'Run level 5 (multi-user with GUI)',
       'Run level 5 (multi-user without GUI)',
       'Run level 3 (Multi-user mode with networking)',
       'Run level 5 (multi-user with display manager)',
       'Run level 3 (Multi User)', 'Run level 5 (Multi User with GUI)',
       'Run level 5 (multi-user graphical)', 'Default',
       'Run level 5 (multi-user with network and display manager)',
       'Run level 5 (multi user mode with GUI)',
       'Run level 5 (Multi User)', 'Run level 3(multi-user)',
       'Run level 3 (Multi-User Mode with Networking)',
       'Run level 5 (Multiuser Networking with GUI)',
       'Run level 3 (Multi-User)',
       'Run level 5 (multi-user, GUI disabled)',
       'Run level 3 (multi user)', 'Run level 5 (multi-user, w/GUI)',
       'Run level 3 (mulit-user)', 'Run level 3',
       'Run level 5 (multi-user w/GUI)', 'Run Level 3 (multi-user)',
       'Run Level 5 (multi-user, graphical)'

In [38]:
c2017_df['SW Avail'].apply(lambda x: re.sub('\t', '', x)).unique()

array(['May-2021', 'Jun-2021', 'Feb-2021', 'Dec-2020', 'Apr-2020',
       'Dec-2016', 'Mar-2022', 'Feb-2022', 'Sep-2021', 'Aug-2021',
       'Apr-2021', 'Jan-2021', 'Mar-2021', 'Aug-2020', 'Aug-2019',
       'Nov-2019', 'May-2019', 'Jun-2019', 'Sep-2018', 'Nov-2018',
       'Dec-2018', 'Mar-2019', 'Feb-2019', 'Mar-2018', 'Feb-2018',
       'Sep-2017', 'Nov-2017', 'Oct-2017', 'May-2017', 'Apr-2019',
       'May-2018', 'Aug-2017', 'Jan-2022', 'Oct-2019', 'Jul-2020',
       'Jul-2016', 'Dec-2019', 'Jul-2021', 'Jun-2020', 'Sep-2019',
       'Jan-2019', 'Aug-2018', 'Nov-2021', 'May-2020', 'Oct-2018',
       'Jan-2018', 'Sep-2020', 'Sep-2016', 'Nov-2020', 'Jul-2018',
       'Nov-2016', 'Dec-2021', 'Apr-2018', 'Jun-2018', 'Oct-2016',
       'Oct-2020', 'Jul-2017', 'Feb-2020', 'Jan-2020', 'Jul-2019',
       'Dec-2017'], dtype=object)